In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

C:\Users\Martin\AppData\Local\Temp\ipykernel_1528\749607990.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def XyData(csv_path):
    usecols = ['content', 'type']
    df = pd.read_csv(csv_path, usecols=usecols)
    df.dropna(subset=['type'], inplace = True) #remove empty labels
    #df.drop(df[df['type'] == 'unknown'].index, inplace=True) #remove 'unknown' label
    df.reset_index(drop=True, inplace=True) #reset index

    y = pd.DataFrame({'result' : np.where(df['type'] == 'fake', 1, 0)}) #[1, 0] = [fake, not fake]

    unique_words = Counter()
    df['content'].str.split().apply(unique_words.update)
    unique_words.subtract(unique_words) #keep keys, reset count
 
    frames = []  
    for row in range(len(df)):
        words = unique_words.copy()
        words.update(df.loc[row,"content"].split())
        frames += [pd.DataFrame(words, index=[row])]
    X = pd.concat(frames)
         
    X.to_csv('X.csv', index=False)
    y.to_csv('y.csv', index=False)

#XyData('..\\finished.csv')
XyData('OLDfinished.csv')

In [19]:
def Split(X_path, y_path):
    df_X = pd.read_csv(X_path)
    df_y = pd.read_csv(y_path)
    X_train, X_test, y_train, y_test = train_test_split(df_X,
                                                        df_y,
                                                        train_size = 0.8,
                                                        random_state = 1,#0,
                                                        shuffle = True,#False,
                                                        stratify = None)

    X_test, X_val, y_test, y_val = train_test_split(X_test,
                                                    y_test,
                                                    test_size= 0.5,
                                                    random_state = 1,#0,
                                                    shuffle = True,#False,
                                                    stratify = None)

    return X_train, X_test, X_val, y_train, y_test, y_val
X_train, X_test, X_val, y_train, y_test, y_val = Split('X.csv', 'y.csv')

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

model = LinearRegression()
fit = model.fit(X_train, y_train['result'])
y_pred = fit.predict(X_test)
y_pred = [round(x,0) for x in y_pred]

mse = mean_squared_error(y_test['result'], y_pred)
print("LinearRegression MSE: ", mse)
acc = accuracy_score(y_test['result'], y_pred)
print("LinearRegression accuracy: ", acc)

LinearRegression MSE:  0.25
LinearRegression accuracy:  0.75


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

model = LogisticRegression(random_state=0, max_iter=1000)
fit = model.fit(X_train, y_train['result'])
y_pred = fit.predict(X_test)
y_pred = [round(x,0) for x in y_pred]

mse = mean_squared_error(y_test['result'], y_pred)
print("LinearRegression MSE: ", mse)
acc = accuracy_score(y_test['result'], y_pred)
print("LinearRegression accuracy: ", acc)

LinearRegression MSE:  0.125
LinearRegression accuracy:  0.875


In [22]:
c = Counter()
c.update(y_test['result'])
print(c[1]/(c[1]+c[0]))

0.6666666666666666
